In [ ]:
from pathlib import Path

import pydicom

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import ndimage

import imageio

from PIL import Image

In [ ]:
df = pd.read_csv('../data/02_intermediate/dmd_index.csv', index_col=0)

In [ ]:
features = ['group', 'treatment', 'center', 'patient', 'slice', 'modality', 'm', 'dz', 'study_no', 'timepoints']

filtered = df[(df.keep) & (df.set == 'train') & (df.treatment == 'CONTROL') & (df.slice == 'mid')][features]

filtered.head(10)

In [ ]:
cine_path = df.loc[5].path
tagg_path = df.loc[7].path

In [ ]:
def get_images(path: str) -> np.ndarray:
    path = Path(path)
    dcm_images = [f for f in path.iterdir() if f.is_file() and f.suffix == ".dcm"]

    res = map(
        lambda ds: (ds.InstanceNumber, ds.pixel_array),
        map(pydicom.dcmread, dcm_images),
    )
    return np.array(list(zip(*sorted(res, key=lambda item: item[0])))[1])

In [ ]:
tagg = get_images(tagg_path)
cine = get_images(cine_path)

print(f'Extracted tagged image {tagg.shape} and according cine {cine.shape}')

zoom_shape = tuple(np.array(tagg.shape) / np.array(cine.shape))

cine = ndimage.zoom(cine, zoom_shape)

print(f'Cine downsampled in time to {cine.shape}')

In [ ]:
save_path = Path('../../figures/gifs/cine-vs-tagged')
save_path.mkdir(parents=True, exist_ok=True)

for t in range(tagg.shape[0]):
    fig, ax = plt.subplots(2, 1, figsize=(3, 5))

    ax[0].imshow(cine[t], cmap='gray')
    ax[1].imshow(tagg[t], cmap='gray')

    for axe in ax:
        axe.set_xticks([])
        axe.set_yticks([])

    plt.tight_layout()
    plt.savefig(save_path / f'time_{t:02}.png', dpi=300)

frames = list(map(imageio.imread, sorted(save_path.iterdir())))

In [ ]:
imageio.mimsave('../../figures/cine_vs_tagged.gif', frames, format='GIF', duration=0.2)

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(7, 4))

for i, t in enumerate([0, 12, 24]):
    ax[0, i].imshow(cine[t], cmap='gray'), ax[0, i].axis('off')
    ax[0, i].set_title(r'$t_{%s}$' % t)
    ax[1, i].imshow(tagg[t], cmap='gray'), ax[1, i].axis('off')

plt.tight_layout()
plt.savefig('../../figures/cine-vs-tagged.pdf', bbox_inches='tight')
plt.show()